In [1]:
from latentneural.lorenz import LorenzGenerator, ic
from latentneural.data import DataManager
from latentneural.utils import logger
import numpy as np
import tensorflow as tf
import os
import yaml
from copy import deepcopy

In [2]:
step = 0.01 # Time step in the simulatio (in seconds)
stop = 1 # Final time for the simulation (in seconds)
neurons = 30 # Number of neurons
base_rate = 5 # Base frequency (in Hz)
latent_dim = 3 # Lorenz latent variables encoded in neural activity (relevant+irrelevant)
relevant_dim = 2 # Lorenz latent variables encoded in behavioural activity (relevant)
behaviour_dim = 4 # Behavioural channels
conditions = 1 # Number of conditions to be tested (each condition has different weight matrices to convert latent variables into firing rates and behaviour)
trials = 1000 # Number of trials to be tested for each condition
initial_conditions = ic.uniform(low=-10, high=10) # Sample initial conditions of Lorenz system from uniform distribution
behaviour_sigma=1 # Standard deviation noise added to behaviour
seed=0

In [3]:
with open(os.path.join('..', 'latentneural', 'data', 'storage', \
                       'lorenz', 'grid', 'settings_template.yaml'), 'r') as f:
   training_settings_template = yaml.safe_load(f)
print(training_settings_template)

{'data': {'behavioural_keys': {'test': 'test_behaviours', 'train': 'train_behaviours', 'validation': 'valid_behaviours'}, 'dataset_filename': 'dataset.h5', 'directory': './latentneural/data/storage/lorenz/grid/{{name}}', 'latent_keys': {'test': 'test_latent', 'train': 'train_latent', 'validation': 'valid_latent'}, 'metadata_filename': 'metadata.json', 'neural_keys': {'test': 'test_data', 'train': 'train_data', 'validation': 'valid_data'}}, 'model': {'settings': {'default_layer_settings': {'kernel_initializer': {'arguments': {'distribution': 'normal', 'mode': 'fan_in', 'scale': 1.0}, 'type': 'variance_scaling'}, 'kernel_regularizer': {'arguments': {'l': 0.1}, 'type': 'l2'}}, 'encoded_space': 64, 'layers': {'encoder': {'dropout': 0.05, 'var_max': 0.1, 'var_min': 0.1}, 'decoder': {'kernel_initializer': {'arguments': {'distribution': 'normal', 'mode': 'fan_in', 'scale': 1.0}, 'type': 'variance_scaling'}, 'kernel_regularizer': {'arguments': {'l': 1}, 'type': 'l2'}, 'original_cell': False, '

In [4]:
np.random.seed(seed)
tf.random.set_seed(seed)

for behaviour_sigma in [0.5, 1, 2]:
    for base_rate in [5, 10, 15]:
        for train_num in [50, 100, 200]:
            inner_seed = np.random.randint(0, 2**32 - 1)
            g: LorenzGenerator = LorenzGenerator()
            time, behaviour, spikes, rates, behaviour_w, neural_w, latent = \
                g.generate_spikes_and_behaviour(
                    step=step,
                    stop=stop,
                    n=neurons,
                    base=base_rate,
                    l=latent_dim,
                    b=relevant_dim,
                    y=behaviour_dim,
                    behaviour_sigma=behaviour_sigma,
                    trials=trials + 2 * train_num,
                    initial_conditions=initial_conditions,
                    conditions=1,
                    seed=inner_seed
                )

            c_i = 0
            generation_settings = dict(
                step=step,
                stop=stop,
                neurons=neurons,
                base_rate=base_rate,
                latent_dim=latent_dim,
                relevant_dim=relevant_dim,
                behaviour_dim=behaviour_dim,
                conditions=conditions,
                behaviour_sigma=behaviour_sigma,
                trials=trials + 2 * train_num,
                initial_conditions={'type': 'uniform', 'arguments': {'min': -10, 'max': 10}},
                selected_condition=c_i,
                seed=inner_seed)

            valid_num = train_num
            test_num = trials

            dataset, settings = DataManager.build_dataset(
                neural_data=spikes[c_i,:,:,:],
                behaviour_data=behaviour[c_i,:,:,:],
                settings=generation_settings,
                noisless_behaviour_data=latent[c_i,:,:,-behaviour_w.shape[1]:] @ behaviour_w[c_i,:,:],
                rates_data=rates[c_i,:,:,:],
                latent_data=latent[c_i,:,:,:],
                time_data=time,
                behaviour_weights=behaviour_w[c_i,:,:],
                neural_weights=neural_w[c_i,:,:],
                train_pct=train_num,
                valid_pct=valid_num,
                test_pct=test_num
            )
            dirname = 'train_trials=%d|baselinerate=%d|behaviour_sigma=%.1f' % \
                (train_num, base_rate, behaviour_sigma)
            data_dir = os.path.join('..', 'latentneural', 'data', 'storage', 'lorenz', 'grid', \
                                    dirname)
            DataManager.store_dataset(
                dataset=dataset,
                settings=settings,
                directory=data_dir)
            logger.info('Dataset stored.')
            
            training_settings = deepcopy(training_settings_template)
            training_settings['data']['directory'] = \
                training_settings['data']['directory'].replace('{{name}}', dirname)
            training_settings['output']['directory'] = \
                training_settings['output']['directory'].replace('{{name}}', dirname)
            # keeping increase per epoch constant (updates are based on batches)
            training_settings['runtime']['weights']['update_rate'][1] = \
                training_settings['runtime']['weights']['update_rate'][1] * 100 / train_num
            training_settings['runtime']['weights']['update_rate'][2] = \
                training_settings['runtime']['weights']['update_rate'][2] * 100 / train_num
            # keeping start epoch constant (start time is based on batches)
            training_settings['runtime']['weights']['update_start'][1] = \
                training_settings['runtime']['weights']['update_start'][1] * train_num / 100
            
            with open(os.path.join(data_dir, 'training_settings.yaml'), 'w') as fp:
                yaml.dump(training_settings, fp)
            logger.info('Training settings stored.')
            logger.info('Stored train_trials=%d|baselinerate=%d|behaviour_sigma=%.1f' % \
                (train_num, base_rate, behaviour_sigma))

[16:14:29.172] INFO [latentneural.utils.logging.store_dataset:185] Saving to ../latentneural/data/storage/lorenz/grid/train_trials=50|baselinerate=5|behaviour_sigma=0.5/dataset.h5
[16:14:29.184] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: train_data
[16:14:29.187] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: valid_data
[16:14:29.189] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: test_data
[16:14:29.209] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: train_behaviours
[16:14:29.211] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: valid_behaviours
[16:14:29.213] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: test_behaviours
[16:14:29.217] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: train_behaviours_noiseless
[16:14:29.218] INFO [latentneural.utils.logging.store

[16:14:48.226] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: valid_behaviours
[16:14:48.228] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: test_behaviours
[16:14:48.233] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: train_behaviours_noiseless
[16:14:48.234] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: valid_behaviours_noiseless
[16:14:48.236] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: test_behaviours_noiseless
[16:14:48.240] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: train_rates
[16:14:48.243] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: valid_rates
[16:14:48.246] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: test_rates
[16:14:48.269] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: train_

[16:15:06.881] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: train_rates
[16:15:06.884] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: valid_rates
[16:15:06.887] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: test_rates
[16:15:06.909] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: train_latent
[16:15:06.910] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: valid_latent
[16:15:06.912] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: test_latent
[16:15:06.915] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: time_data
[16:15:06.916] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: relevant_dims
[16:15:06.918] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: behaviour_weights
[16:15:06.920] INFO [latentneural.utils

[16:15:25.067] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: time_data
[16:15:25.068] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: relevant_dims
[16:15:25.070] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: behaviour_weights
[16:15:25.071] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: neural_weights
[16:15:25.073] INFO [latentneural.utils.logging.<module>:66] Dataset stored.
[16:15:25.081] INFO [latentneural.utils.logging.<module>:84] Training settings stored.
[16:15:25.082] INFO [latentneural.utils.logging.<module>:85] Stored train_trials=50|baselinerate=5|behaviour_sigma=1.0
[16:15:30.911] INFO [latentneural.utils.logging.store_dataset:185] Saving to ../latentneural/data/storage/lorenz/grid/train_trials=100|baselinerate=5|behaviour_sigma=1.0/dataset.h5
[16:15:30.914] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: train_dat

[16:15:43.381] INFO [latentneural.utils.logging.<module>:85] Stored train_trials=50|baselinerate=10|behaviour_sigma=1.0
[16:15:50.000] INFO [latentneural.utils.logging.store_dataset:185] Saving to ../latentneural/data/storage/lorenz/grid/train_trials=100|baselinerate=10|behaviour_sigma=1.0/dataset.h5
[16:15:50.002] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: train_data
[16:15:50.006] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: valid_data
[16:15:50.009] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: test_data
[16:15:50.030] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: train_behaviours
[16:15:50.031] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: valid_behaviours
[16:15:50.033] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: test_behaviours
[16:15:50.037] INFO [latentneural.utils.logging.stor

[16:16:08.123] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: train_behaviours
[16:16:08.125] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: valid_behaviours
[16:16:08.126] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: test_behaviours
[16:16:08.130] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: train_behaviours_noiseless
[16:16:08.132] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: valid_behaviours_noiseless
[16:16:08.133] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: test_behaviours_noiseless
[16:16:08.137] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: train_rates
[16:16:08.141] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: valid_rates
[16:16:08.144] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: 

[16:16:27.204] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: test_behaviours_noiseless
[16:16:27.209] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: train_rates
[16:16:27.212] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: valid_rates
[16:16:27.216] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: test_rates
[16:16:27.238] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: train_latent
[16:16:27.240] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: valid_latent
[16:16:27.242] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: test_latent
[16:16:27.245] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: time_data
[16:16:27.246] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: relevant_dims
[16:16:27.248] INFO [latentneur

[16:16:45.294] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: test_latent
[16:16:45.297] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: time_data
[16:16:45.299] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: relevant_dims
[16:16:45.300] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: behaviour_weights
[16:16:45.302] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: neural_weights
[16:16:45.304] INFO [latentneural.utils.logging.<module>:66] Dataset stored.
[16:16:45.311] INFO [latentneural.utils.logging.<module>:84] Training settings stored.
[16:16:45.312] INFO [latentneural.utils.logging.<module>:85] Stored train_trials=100|baselinerate=10|behaviour_sigma=2.0
[16:16:52.305] INFO [latentneural.utils.logging.store_dataset:185] Saving to ../latentneural/data/storage/lorenz/grid/train_trials=200|baselinerate=10|behaviour_sigma=2.0/datas

[16:17:05.602] INFO [latentneural.utils.logging.<module>:84] Training settings stored.
[16:17:05.603] INFO [latentneural.utils.logging.<module>:85] Stored train_trials=100|baselinerate=15|behaviour_sigma=2.0
[16:17:13.284] INFO [latentneural.utils.logging.store_dataset:185] Saving to ../latentneural/data/storage/lorenz/grid/train_trials=200|baselinerate=15|behaviour_sigma=2.0/dataset.h5
[16:17:13.287] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: train_data
[16:17:13.294] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: valid_data
[16:17:13.305] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: test_data
[16:17:13.328] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: train_behaviours
[16:17:13.331] INFO [latentneural.utils.logging.store_dataset:204] Saving variable with name: valid_behaviours
[16:17:13.336] INFO [latentneural.utils.logging.store_dataset:204] Saving 